In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import glob
from keras.callbacks import TensorBoard
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.layers.core import RepeatVector, Permute
from keras.models import Sequential, Model
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize


Using TensorFlow backend.


In [2]:
data = glob.glob('./data_faces/Train/*.jpg')
# img = load_img('./data_faces/Train/0A9kTN.jpg')

data = data[:2000]
print('Len data: ', len(data))
images = []
for items in data:
    images.append(img_to_array(load_img(items)))
images = np.array(images, dtype=float)

Len data:  2000


In [3]:
print('Len images: ', len(images))
print('Image shape: ', images[0].shape)

Len images:  2000
Image shape:  (256, 256, 3)


#### 1.0/255 : Indicates that we are using a 24-bit RGB color space. 0-255 for each color channel

In [4]:
X_train = 1.0/255*images
print(X_train.shape)

(2000, 256, 256, 3)


#### Load weights from InceptionResNetV2
InceptionResNetV2 is the most powerful classifier. Extract the classification layer and merge it with the output from the encoder

In [5]:
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.get_default_graph()

# MODEL

Our model is the next one:
https://github.com/baldassarreFe/deep-koalarization

<img src="full_network.png">

In [6]:
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 

#Decoder
decoder_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 256, 256, 1)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 128, 128, 64)  640         input_2[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 128, 128, 128) 73856       conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_3 (Conv2D)                (None, 64, 64, 128)   147584      conv2d_2[0][0]                   
___________________________________________________________________________________________

First, we have to resize the image to fit into the Inception model. Then we use the preprocessor to format the pixel and color values according to the model. In the final step, we run it through the Inception network and extract the final layer of the model.

In [7]:
# grayscaled_rgb are images
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    # Resize the images to (299, 299, 3), the Inception model works with this
    # dimension
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    # Conver to array
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    # Preprocess the array to make it with this dimension (sample, 299, 299, 3)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    # Predict these images in Inception Model
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

In [8]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

In [9]:
#Generate training data
batch_size = 50

# We change from RGB to LAB:
# - L: lightness
# - a: green-red
# - b: blue-yellow

def image_a_b_gen(batch_size):
    for batch in datagen.flow(X_train, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)


In [10]:
tensorboard = TensorBoard(log_dir="log/")

model.compile(optimizer='rmsprop', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], 
                    epochs=21, steps_per_epoch=90)

# Save model# Save  
model_json = model.to_json()
with open("./Models/model_full_colorizer.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./Models/model_full_colorizer.h5")

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/21
90/90 [==============================] - 473s - loss: 0.0691   
Epoch 2/21
90/90 [==============================] - 452s - loss: 0.0114   
Epoch 3/21
90/90 [==============================] - 460s - loss: 0.0114   
Epoch 4/21
90/90 [==============================] - 466s - loss: 0.0135   
Epoch 5/21
90/90 [==============================] - 461s - loss: 0.0106   
Epoch 6/21
90/90 [==============================] - 457s - loss: 0.0107   
Epoch 7/21
90/90 [==============================] - 455s - loss: 0.0107   
Epoch 8/21
90/90 [==============================] - 455s - loss: 0.0105   
Epoch 9/21
90/90 [==============================] - 456s - loss: 0.0106   
Epoch 10/21
90/90 [==============================] - 459s - loss: 0.0104   
Epoch 11/21
90/90 [==============================] - 456s - loss: 0.0117   
Epoch 12/21
90/90 [==============================] - 455s - loss: 0.0104   
Epoch 13/21
90/90 [=====

In [13]:
color_me = []
for filename in os.listdir('./data_faces/Test/'):
    color_me.append(img_to_array(load_img('./data_faces/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations:
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("./results_full/img_"+str(i)+".png", lab2rgb(cur))

C:\Users\aleja\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\aleja\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: ./results_full/img_30.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\aleja\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: ./results_full/img_236.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\aleja\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: ./results_full/img_383.png is a low contrast image
  warn('%s is a low contrast image' % fname)
